We want to explore a first solution guided by common rules of thumb when building CNNs.

# Load Data for first Training
For our first attempt we want to use the full dataset without considering the possibility of being lucky or having bad luck. 

The final evaluations will be performed via k-Fold-CV.

In [1]:
import models.structure.base_model_wrapper
from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper
from torch.utils.data import DataLoader
from dataset.dataset_loader import dataset_loader

train, test = dataset_loader((224, 224), is_grayscale=False)

# We take 20% of dataset as validation.
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test, batch_size=32, shuffle=True)

In [2]:
print(f"Validation set is {len(validation) / (len(train) + len(test)) * 100:.2f}% of total dataset.")
print(f"This might be too little to realistically do some early stopping. Estimating epoches might be the way to go.")

# Handmade First Attempt
The first handmade attempt is done before fine tuning the hyperparameters. We simply follow,
for this step, common rules of thumb such as:
- The number of filter may be mixed in increasing order to better match more complex patterns in the images
- A (3x3) kernel generally performs well
- CNN benefit from having a multiple succession of layers

In [3]:
from models.simple_cnn.hand_tailored_conv_net import HandTailoredConvNetV1
from models.structure.learning_parameters.sgd_learning_parameters import SgdLearningParameters

model_family = HandTailoredConvNetV1()
model = HandTailoredConvNetV1().make_model((3, 224, 224))

learning_parameters = SgdLearningParameters(learning_rate=1e-4)
learning_parameters.compile_model(model)

model.summary()

In [22]:
import keras

keras.utils.plot_model(
    model, to_file='hand_tailored_conv_net.png', show_layer_names=True, expand_nested=True, show_shapes=True
)

In [4]:
import keras

# In this simple approach we have fixed [Train 0.8 - Val 0.2 - Test] without doing any k-fold CV.
# The number of epoches is fixed by early stopping. (In the case of k-fold-CV it will be a hp given by average performance?)
history = model.fit(x=train_dataloader, validation_data=validation_dataloader, batch_size=32, epochs=120, callbacks=[
    # To avoid going further when training
    keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=1e-4, patience=10,
        verbose=1, mode='min', restore_best_weights=True
    ),
    # To persist the history
    keras.callbacks.CSVLogger(f"hand-tailored-cnn.log", separator=",", append=True)
])

In [9]:
res = model.evaluate(test_dataloader)
persist_model: bool = False

# We don't want to persist the model as I already saved it.
if persist_model:
    model.save('hand_tailored_conv_net.keras')

In [20]:
print(f"Test accuracy is {res[1] * 100:.2f}% while loss is {res[0]}")
print(f"The model is not that bad considering we have no pre-processing done and the parameters were chosen freely")

In [2]:
import pandas

csv = pandas.read_csv(f"hand-tailored-cnn.log")

In [3]:
import plotly.express as px

loss_figure = px.line(csv, x="epoch", y=["loss", "val_loss"], template="plotly_white", markers=True)
loss_figure.update_layout(title="Loss", xaxis_title="Epoch", yaxis_title="Loss")

In [4]:
figure = px.line(csv, x="epoch", y=["accuracy", "val_accuracy"], template="plotly_white", markers=True)
figure.update_layout(title="Accuracy", xaxis_title="Epoch", yaxis_title="Loss")

The model I proposed works fairly well although it seems to be overfitting. 

The chosen hyperparameters seem to be good enough, but we will still try to find the best network structure via Keras Tuner.
 
The model we obtained might be the result of a lucky run. We should run k-fold cv to really estimate the model performance.

# K-fold CV


In [ ]:
# How do I choose the number of epochs?
# Nested k-fold-CV

# Reduce the parameters to avoid overfitting or add regularization?

In [ ]:
class HandTailoredConvV2(models.structure.base_model_wrapper.BaseModelWrapper):
    def make_layers(self, input_shape: (int, int, int)) -> tuple[keras.Layer, keras.Layer]:
        input_layer = keras.Input(shape=input_shape, name=self.__class__.__name__)
        x = keras.layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu",
                                data_format=self.data_format.value)(input_layer)
        x = keras.layers.MaxPool2D(pool_size=(2, 2), data_format=self.data_format.value)(x)
        
        x = keras.layers.Conv2D(64, kernel_size=(3, 3), padding="same",
                                data_format=self.data_format.value, activation="relu")(x)
        x = keras.layers.MaxPool2D(pool_size=(2, 2), data_format=self.data_format.value)(x)


        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(units=64, activation="relu")(x)
        output_layer = keras.layers.Dense(units=1, activation="sigmoid")(x)
        
        return input_layer, output_layer